Hehe

## Setup

Please run the cells below to install the necessary dependencies.


In [1]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [2]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [3]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

**IMPORTANT : Restart your run-time first and then run the cells below.**

## Inference


In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [2]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

### English to Indic Example


In [7]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

spiderman_sents = [
    "Spider-Man woke up late because his alarm clock got stuck in his web.",
    "While brushing his teeth, he accidentally shot a web and got stuck to the ceiling.",
    "Aunt May asked why there was toothpaste on the roof, and he had no answer.",
    "He tried swinging to work, but forgot his web-shooters at home.",
    "So, he had to take the subway like a normal person, in full costume.",
    "People kept asking for selfies, and one kid asked if he was a birthday clown.",
    "At the Daily Bugle, Jonah Jameson yelled, 'You're late!' even before he entered.",
    "Suddenly, a villain appeared—but slipped on a banana peel before doing anything evil.",
    "Spider-Man helped him up and gave him a juice box to calm down.",
    "At the end of the day, Spidey saved the city... and also fixed Aunt May’s toaster, which he somehow broke in the morning.",
]


src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(spiderman_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(spiderman_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



eng_Latn - hin_Deva
eng_Latn: Spider-Man woke up late because his alarm clock got stuck in his web.
hin_Deva: स्पाइडर-मैन देर से उठा क्योंकि उसकी अलार्म घड़ी उसके जाल में अटक गई थी। 
eng_Latn: While brushing his teeth, he accidentally shot a web and got stuck to the ceiling.
hin_Deva: अपने दांतों को ब्रश करते समय, उन्होंने गलती से एक जाल को गोली मार दी और छत पर फंस गए। 
eng_Latn: Aunt May asked why there was toothpaste on the roof, and he had no answer.
hin_Deva: चाची मे ने पूछा कि छत पर टूथपेस्ट क्यों था, और उनके पास कोई जवाब नहीं था। 
eng_Latn: He tried swinging to work, but forgot his web-shooters at home.
hin_Deva: उन्होंने काम पर जाने की कोशिश की, लेकिन घर पर अपने वेब-शूटर्स को भूल गए। 
eng_Latn: So, he had to take the subway like a normal person, in full costume.
hin_Deva: इसलिए, उन्हें पूरी पोशाक में एक सामान्य व्यक्ति की तरह मेट्रो लेना पड़ा। 
eng_Latn: People kept asking for selfies, and one kid asked if he was a birthday clown.
hin_Deva: लोग सेल्फी लेने के लिए पूछते रहे, और 

### Indic to English Example

In [5]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

nightwing_sents = [
    "नाइटविंग कभी बैटमैन का साथी था, लेकिन अब वह खुद एक महान हीरो बन चुका है।",
    "गॉथम सिटी की गलियों में नाइटविंग की मौजूदगी से अपराधियों में डर बना रहता है।",
    "जब डिक ग्रेसन ने रोबिन की पहचान छोड़ी, तब उसने नाइटविंग के रूप में नई शुरुआत की।",
    "नाइटविंग न केवल ताकतवर है, बल्कि वह बहुत बुद्धिमान और रणनीतिक भी है।",
    "वह हमेशा न्याय के लिए लड़ता है, चाहे हालात कितने भी मुश्किल क्यों न हों।",
    "ब्लू हेवन में रहते हुए, नाइटविंग ने कई लोगों की जान बचाई और भरोसे का प्रतीक बन गया।",
    "वह अपनी जिम्नास्टिक क्षमता और एक्रोबैटिक स्किल्स से दुश्मनों को चौंका देता है।",
    "नाइटविंग अपने दोस्तों के लिए हमेशा खड़ा रहता है, चाहे कोई भी खतरा सामने क्यों न हो।",
    "डिक ग्रेसन ने अपने अतीत से सीख लेकर खुद को एक बेहतर नायक में ढाला।",
    "नाइटविंग का नीला और काला सूट उसकी पहचान बन चुका है, जो उसे दूसरों से अलग बनाता है।",
]

src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(nightwing_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(nightwing_sents, en_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_en_tokenizer, indic_en_model

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



hin_Deva - eng_Latn
hin_Deva: नाइटविंग कभी बैटमैन का साथी था, लेकिन अब वह खुद एक महान हीरो बन चुका है।
eng_Latn: Nightwing was once Batman's sidekick, but he has now become a legendary hero himself.
hin_Deva: गॉथम सिटी की गलियों में नाइटविंग की मौजूदगी से अपराधियों में डर बना रहता है।
eng_Latn: Nightwing's presence on the streets of Gotham City keeps criminals in fear.
hin_Deva: जब डिक ग्रेसन ने रोबिन की पहचान छोड़ी, तब उसने नाइटविंग के रूप में नई शुरुआत की।
eng_Latn: When Dick Grayson left Robin's identity, he started anew as Nightwing.
hin_Deva: नाइटविंग न केवल ताकतवर है, बल्कि वह बहुत बुद्धिमान और रणनीतिक भी है।
eng_Latn: Not only is Nightwing powerful, but he is also very intelligent and strategic.
hin_Deva: वह हमेशा न्याय के लिए लड़ता है, चाहे हालात कितने भी मुश्किल क्यों न हों।
eng_Latn: He always fights for justice, no matter how difficult the circumstances.
hin_Deva: ब्लू हेवन में रहते हुए, नाइटविंग ने कई लोगों की जान बचाई और भरोसे का प्रतीक बन गया।
eng_Latn: While in Blue Hea

### Indic to Indic Example


In [6]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

spiderman_sents = [
    "स्पाइडर-मैन का असली नाम पीटर पार्कर है, जो एक साधारण लड़का था।",
    "एक रेडियोएक्टिव मकड़ी के काटने से उसे असाधारण शक्तियाँ मिल गईं।",
    "वह दीवारों पर चढ़ सकता है और अपने वेब से दुश्मनों को पकड़ लेता है।",
    "स्पाइडर-मैन हमेशा कहता है: 'बड़ी ताकत के साथ बड़ी जिम्मेदारी भी आती है।'",
    "वह न्यू यॉर्क सिटी की इमारतों के बीच झूलते हुए अपराध से लड़ता है।",
    "चाहे उसकी खुद की ज़िंदगी मुश्किलों से भरी हो, वह दूसरों की मदद करना नहीं छोड़ता।",
    "स्पाइडर-मैन ने कई बार अपने सबसे बड़े दुश्मनों से अकेले मुकाबला किया है।",
    "पीटर पार्कर एक फोटोग्राफर है और अपने जीवन में संतुलन बनाए रखने की कोशिश करता है।",
    "स्पाइडर-मैन की कहानियाँ हमें दिखाती हैं कि एक आम इंसान भी असाधारण बन सकता है।",
    "उसका लाल और नीला सूट, उसकी पहचान और साहस का प्रतीक है।",
]

src_lang, tgt_lang = "hin_Deva", "mar_Deva"
mr_translations = batch_translate(spiderman_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(spiderman_sents, mr_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_indic_tokenizer, indic_indic_model

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



hin_Deva - mar_Deva
hin_Deva: स्पाइडर-मैन का असली नाम पीटर पार्कर है, जो एक साधारण लड़का था।
mar_Deva: स्पायडर-मॅनचे खरे नाव पीटर पार्कर आहे, जो एक साधा मुलगा होता. 
hin_Deva: एक रेडियोएक्टिव मकड़ी के काटने से उसे असाधारण शक्तियाँ मिल गईं।
mar_Deva: किरणोत्सर्गी कोळीच्या दंशामुळे त्याला विलक्षण शक्ती मिळाल्या. 
hin_Deva: वह दीवारों पर चढ़ सकता है और अपने वेब से दुश्मनों को पकड़ लेता है।
mar_Deva: तो भिंतींवर चढू शकतो आणि त्याच्या जाळ्याने शत्रूंना पकडू शकतो. 
hin_Deva: स्पाइडर-मैन हमेशा कहता है: 'बड़ी ताकत के साथ बड़ी जिम्मेदारी भी आती है।'
mar_Deva: स्पायडर-मॅन नेहमीच म्हणतोः'मोठ्या सामर्थ्यासह मोठी जबाबदारी येते'. 
hin_Deva: वह न्यू यॉर्क सिटी की इमारतों के बीच झूलते हुए अपराध से लड़ता है।
mar_Deva: तो न्यूयॉर्क शहरातील इमारतींमध्ये फिरत असताना गुन्हेगारीशी लढतो. 
hin_Deva: चाहे उसकी खुद की ज़िंदगी मुश्किलों से भरी हो, वह दूसरों की मदद करना नहीं छोड़ता।
mar_Deva: त्याचे स्वतःचे जीवन अडचणींनी भरलेले असले तरी तो इतरांना मदत करणे सोडत नाही. 
hin_Deva: स्पाइडर-मैन ने कई बार अपने सबसे बड